<a href="https://colab.research.google.com/github/cedkim/deeplearning-courseenv/blob/master/3-tensorflow-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Neural Network for MNIST Classification

## Import the relevant packages

In [ ]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [ ]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


## Model

### Outline the model

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

### Choose the optimizer and the loss function

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Train

In [ ]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.3347 - accuracy: 0.9048 - val_loss: 0.1596 - val_accuracy: 0.9525
Epoch 2/5
540/540 - 10s - loss: 0.1379 - accuracy: 0.9589 - val_loss: 0.0999 - val_accuracy: 0.9707
Epoch 3/5
540/540 - 10s - loss: 0.0954 - accuracy: 0.9717 - val_loss: 0.0875 - val_accuracy: 0.9772
Epoch 4/5
540/540 - 10s - loss: 0.0743 - accuracy: 0.9771 - val_loss: 0.0634 - val_accuracy: 0.9818
Epoch 5/5
540/540 - 10s - loss: 0.0585 - accuracy: 0.9822 - val_loss: 0.0543 - val_accuracy: 0.9838


## Test the model

In [12]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 13ms/step - loss: 0.0896 - accuracy: 0.9730
